In [1]:
import sys
import os
from tqdm import tqdm

# Aggiungi il percorso del progetto al PYTHONPATH
sys.path.append(os.path.abspath(r"C:\Users\cical\Documents\GitHub\Repositories\pig_tissue_segmentation"))

import src.data.make_dataset as make_dataset
import src.data.image_info as image_info

slice_dict = make_dataset.load_image_slices(r"C:\Users\cical\Documents\GitHub\Repositories\pig_tissue_segmentation\data\raw")

for slice_id in tqdm(slice_dict):
    imageInfo = image_info.ImageSlice(slice_id, slice_dict[slice_id]['wga'], slice_dict[slice_id]['collagen'], slice_dict[slice_id]['autofluorescence'])
    imageInfo.analyse_image()



  0%|          | 0/6 [00:00<?, ?it/s]c:\Users\cical\Documents\GitHub\Repositories\pig_tissue_segmentation\.venv\lib\site-packages\PIL\Image.py:3402: DecompressionBombWarning: Image size (113209979 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
  0%|          | 0/6 [00:01<?, ?it/s]


AttributeError: module 'src.data.segmentation' has no attribute 'watershed_segmentation'